In [1]:
# ETAPE 1 - MERGER LES DIFFERENTS FICHIERS DES 9 FRAMEWORK PROGRAMME

import pandas as pd
from pathlib import Path
from unidecode import unidecode

# === CONFIGURATION ===
raw_dir = Path(r"C:\Users\Leo\Desktop\CORDIS\RAW")
result_dir = Path(r"C:\Users\Leo\Desktop\CORDIS\RESULTS")
fp_versions = ['FP1', 'FP2', 'FP3', 'FP4', 'FP5', 'FP6', 'FP7', 'H2020', 'HORIZON']

org_paths = [raw_dir / version / "organization.csv" for version in fp_versions]
proj_paths = [raw_dir / version / "project.csv" for version in fp_versions]
topics_paths = [raw_dir / version / "topics.csv" for version in fp_versions]
euroscivoc_paths = [raw_dir / version / "euroSciVoc.csv" for version in fp_versions]
legal_paths = [raw_dir / version / "legalBasis.csv" for version in fp_versions]

# === FONCTIONS ===
def load_dataset(path):
    if path.exists():
        try:
            return pd.read_csv(path, sep=None, engine='python', on_bad_lines='skip')
        except UnicodeDecodeError:
            return pd.read_csv(path, sep=None, engine='python', encoding='ISO-8859-1', on_bad_lines='skip')
    else:
        return pd.DataFrame()

def clean_dataframe(df, key_column):
    df = df.copy()
    if key_column in df.columns:
        df = df[df[key_column].notna()]
        df[key_column] = df[key_column].astype(str).str.strip()
    return df

def aggregate_by_project(df, key, value_cols):
    if df.empty:
        return pd.DataFrame(columns=[key] + value_cols)
    df[key] = df[key].astype(str).str.strip()
    return df.groupby(key)[value_cols].agg(lambda x: '; '.join(sorted(set(x.dropna().astype(str))))).reset_index()

def process_fp(fp_idx, version):
    org = clean_dataframe(load_dataset(org_paths[fp_idx]), "projectID")
    proj = clean_dataframe(load_dataset(proj_paths[fp_idx]), "id")
    topics = aggregate_by_project(load_dataset(topics_paths[fp_idx]), "projectID", ["title"])
    scivoc = aggregate_by_project(load_dataset(euroscivoc_paths[fp_idx]), "projectID", ["euroSciVocTitle"])
    legal = aggregate_by_project(load_dataset(legal_paths[fp_idx]), "projectID", ["legalBasis"])

    enriched = org
    if not proj.empty:
        enriched = enriched.merge(proj, left_on="projectID", right_on="id", how="left")
    if not topics.empty:
        enriched = enriched.merge(topics, on="projectID", how="left")
    if not scivoc.empty:
        enriched = enriched.merge(scivoc, on="projectID", how="left")
    if not legal.empty:
        enriched = enriched.merge(legal, on="projectID", how="left")

    enriched['frameworkProgramme'] = version.upper()

    out_path = result_dir / f"Enriched_{version}.csv"
    enriched.to_csv(out_path, index=False)
    print(f"Exporté : {out_path}")
    return enriched

def clean_and_normalize(text):
    if pd.isna(text):
        return ''
    return unidecode(text).lower().strip()

# === TRAITEMENT GLOBAL ===
all_data = []

for i, version in enumerate(fp_versions):
    enriched_df = process_fp(i, version)
    all_data.append(enriched_df)

if all_data:
    full_df = pd.concat(all_data, ignore_index=True)

    # === FILTRAGE : supprimer les lignes sans 'name' ni 'shortName'
    print("Avant filtrage :", full_df.shape)
    full_df = full_df[~(
        (full_df['name'].fillna('').str.strip() == '') &
        (full_df['shortName'].fillna('').str.strip() == '')
    )]
    print("Après filtrage :", full_df.shape)

    # === AJOUT DE LA COLONNE NORMALISÉE 'namekey'
    full_df['namekey'] = full_df['name'].fillna('').apply(clean_and_normalize)

    # === EXPORT DES DONNÉES NETTOYÉES ===
    final_path = result_dir / "R1_FP.csv"
    full_df.to_csv(final_path, index=False)
    print(f"Export global nettoyé : {final_path}")

Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP1.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP2.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP3.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP4.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP5.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP6.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_FP7.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_H2020.csv
Exporté : C:\Users\Leo\Desktop\CORDIS\RESULTS\Enriched_HORIZON.csv
Avant filtrage : (699912, 49)
Après filtrage : (699212, 49)
Export global nettoyé : C:\Users\Leo\Desktop\CORDIS\RESULTS\R1_FP.csv


In [3]:
# ETAPE 2 - IDENTIFIER LES SERVICES GEOLOGIQUES PAR LES RACINES "geo" / "geol" DANS "namekey"

import pandas as pd
from pathlib import Path

# === CONFIGURATION ===
result_dir = Path(r"C:\Users\Leo\Desktop\CORDIS\RESULTS")
input_file = result_dir / "R1_FP.csv"
output_file = result_dir / "R2_geo_unique.csv"

# === CHARGEMENT DES DONNÉES ===
df = pd.read_csv(input_file, low_memory=False)

# === FILTRAGE : ORGANISATIONS CONTENANT "geo" OU "geol" DANS namekey ===
filtered_df = df[df['namekey'].str.contains(r'geo|geol', case=False, na=False)]

# === EXTRACTION DES ENTRÉES UNIQUES PAR namekey ===
unique_geol_df = df[df['namekey'].isin(filtered_df['namekey'])].drop_duplicates(subset='namekey')

# === EXPORT DU RÉSULTAT FILTRÉ ===
unique_geol_df.to_csv(output_file, index=False)
print(f"\nExport effectué : {output_file}")
print(unique_geol_df.shape)


Export effectué : C:\Users\Leo\Desktop\CORDIS\RESULTS\R2_geo_unique.csv
(1257, 50)


In [5]:
# ETAPE 5 : DÉTECTER LES SERVICES GÉOLOGIQUES DANS LA BASE POUR LES UNIFORMISER

import pandas as pd
from pathlib import Path
from unidecode import unidecode

# === Chargement des fichiers ===
base = Path(r"C:\Users\Leo\Desktop\CORDIS\RESULTS")
df_main = pd.read_csv(base / "R1_FP.csv", encoding='utf-8-sig', low_memory=False)
df_geol = pd.read_csv(base / "R3_repertoire_GS.csv", encoding='utf-8-sig', delimiter=';', low_memory=False)

# === Normalisation des champs utiles ===
def normalize(s):
    return unidecode(str(s).strip().upper()) if pd.notna(s) else None

# namekey et shortname doivent être normalisés (résultat du test précédent)
df_main['namekey_norm'] = df_main['namekey'].apply(normalize)
df_main['shortname_norm'] = df_main['shortName'].apply(normalize)
df_main['country_norm'] = df_main['country']  # pas besoin de normaliser, déjà propre

df_geol['namekey_norm'] = df_geol['namekey'].apply(normalize)
df_geol['shortname_norm'] = df_geol['shortname'].apply(normalize)
df_geol['country_norm'] = df_geol['country'].apply(normalize)

# === Étape 1 : Détection par namekey exact ===
set_namekeys = set(df_geol['namekey_norm'].dropna())
df_main['GS'] = df_main['namekey_norm'].isin(set_namekeys).astype(int)
df_main['GS_detected_by'] = df_main['GS'].apply(lambda x: 'namekey' if x == 1 else '')

# === Étape 2 : Détection par (shortname + country) si GS == 0 ===
geo_acronyms = set(
    zip(
        df_geol.loc[df_geol['shortname_norm'].notna(), 'shortname_norm'],
        df_geol.loc[df_geol['shortname_norm'].notna(), 'country_norm']
    )
)

mask = df_main['GS'] == 0
matches = df_main[mask].apply(
    lambda row: (row['shortname_norm'], row['country_norm']) in geo_acronyms,
    axis=1
)

df_main.loc[mask & matches, 'GS'] = 1
df_main.loc[mask & matches, 'GS_detected_by'] = 'shortname+country'

# === Export final ===
output_path = base / "R5_all_GS_detected.csv"
df_main.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
print(f"Export terminé : {output_path}")

# === Résumé des méthodes de détection ===
print("\nMéthodes de détection des GS :")
print(df_main['GS_detected_by'].value_counts(dropna=False))

Export terminé : C:\Users\Leo\Desktop\CORDIS\RESULTS\R5_all_GS_detected.csv

Méthodes de détection des GS :
GS_detected_by
                     697860
namekey                1319
shortname+country        33
Name: count, dtype: int64


In [7]:
#STEP 6: TROUVER LES PARTENAIRES DE PROJETS DES SERVICES GEOLOGIQUES DANS LA BASE FP

import pandas as pd
from pathlib import Path

# Chemin vers les résultats
result_dir = Path(r"C:\Users\Leo\Desktop\CORDIS\RESULTS")

# Charger les fichiers nécessaires
file_organisations = result_dir / "R1_FP.csv"
file_geological_surveys = result_dir / "R5_all_GS_detected.csv"

# Lire les données
df_orgs = pd.read_csv(file_organisations, encoding='utf-8-sig', low_memory=False)
df_geo_orgs = pd.read_csv(file_geological_surveys, encoding='utf-8-sig', delimiter=';', low_memory=False)

# Ne garder que les lignes identifiées comme GS
df_geo_orgs = df_geo_orgs[df_geo_orgs['GS'] == 1]

# Définir les groupes de programmes
fp_groups = {
    'FP1_FP6': ('rcn_y', ['FP1', 'FP2', 'FP3', 'FP4', 'FP5', 'FP6']),
    'FP7_H2020': ('id', ['FP7', 'H2020', 'HORIZON'])
}

# Extraire les partenaires
df_partners = []

for label, (key, fps) in fp_groups.items():
    df_geo_subset = df_geo_orgs[df_geo_orgs['frameworkProgramme'].isin(fps)]
    geo_ids = df_geo_subset[key].dropna().unique()
    df_fp_partners = df_orgs[
        (df_orgs['frameworkProgramme'].isin(fps)) &
        (df_orgs[key].isin(geo_ids))
    ]
    df_partners.append(df_fp_partners)

# Concaténer tous les résultats
df_all_geo_partners = pd.concat(df_partners + [df_geo_orgs], ignore_index=True)

# Sauvegarder le fichier final
output_path = result_dir / "R6_GS_and_partners.csv"
df_all_geo_partners.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

print(f"Total : {df_all_geo_partners.shape[0]}")

Total : 13918


In [9]:
#STEP 7: STANDARDISER LES SERVICES GEOLOGIQUES A PARTIR DU REFERENIEL (STEP 4. "R4_GS_root.csv")

import pandas as pd
from pathlib import Path
from unidecode import unidecode

# === Fonction de normalisation ===
def normalize(s):
    return unidecode(str(s).strip().upper()) if pd.notna(s) else None

# === Chargement des fichiers ===
base = Path(r"C:\Users\Leo\Desktop\CORDIS\RESULTS")
df_main = pd.read_csv(base / "R6_GS_and_partners.csv", encoding='utf-8-sig', sep=';', low_memory=False)
df_ref = pd.read_csv(base / "R4_GS_root.csv", encoding='utf-8-sig', sep=';')

# === Normalisation des pays pour jointure ===
df_main['country_norm'] = df_main['country'].apply(normalize)
df_ref['country_norm'] = df_ref['country'].apply(normalize)

# === Jointure avec le référentiel (ajout des données officielles) ===
df_merged = df_main.merge(
    df_ref.add_prefix("GS_"),
    how='left',
    left_on='country_norm',
    right_on='GS_country_norm'
)

# === Liste des champs à harmoniser depuis le référentiel ===
fields_to_harmonize = ['name', 'shortname', 'street', 'postcode', 'city', 'latitude', 'longitude']

# === Harmonisation conditionnelle (uniquement si GS détecté) ===
for field in fields_to_harmonize:
    df_merged.loc[df_merged['GS'] == 1, field] = df_merged.loc[df_merged['GS'] == 1, f'GS_{field}']

# === Export du résultat harmonisé ===
output_path = base / "R7_GS_harmonized_and_partners.csv"
df_merged.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

# === Résumé du traitement ===
print("Lignes harmonisées :", (df_merged['GS'] == 1).sum())
print("Colonnes ajoutées :", [f'GS_{col}' for col in fields_to_harmonize])

Lignes harmonisées : 1352
Colonnes ajoutées : ['GS_name', 'GS_shortname', 'GS_street', 'GS_postcode', 'GS_city', 'GS_latitude', 'GS_longitude']
